<a href="https://colab.research.google.com/github/mervefilizbaker1/DRIVER-RISK-PREDICTION/blob/main/drp_unsupervised.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### DRIVER RISK PREDICTION - UNSUPERVISED LEARNING


In [2]:
#libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

In [3]:
df_accident = pd.read_csv('/content/drive/MyDrive/DRP/df_accident')

In [4]:
X = df_accident.drop(['risk_class','Accident_severity','driver_risk_score','env_risk_score','behaviour_risk_score','total_risk_score'], axis=1)
y = df_accident['risk_class']

In [5]:
df_accident.head()

,Accident_severity,Age_band_of_driver_31-50,Age_band_of_driver_Over 51,Age_band_of_driver_Under 18,Age_band_of_driver_Unknown,Sex_of_driver_Male,Sex_of_driver_Unknown,Educational_level_Elementary school,Educational_level_High school,Educational_level_Illiterate,...,Cause_of_accident_Overspeed,Cause_of_accident_Overtaking,Cause_of_accident_Overturning,Cause_of_accident_Turnover,Cause_of_accident_Unknown,driver_risk_score,env_risk_score,behaviour_risk_score,total_risk_score,risk_class
0,2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.475,0.200,0.5,0.391667,Low
1,2,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.350,0.200,1.0,0.516667,Medium
2,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.475,0.200,0.2,0.291667,Low
3,2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.425,0.650,0.2,0.425000,Medium
4,2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.425,0.425,1.0,0.616667,Medium


In [6]:
df_accident['risk_class'].value_counts()

,count
risk_class,
Medium,6307
Low,5802
High,207


# PCA